# Evolutionary Computation - Assignment 1: Greedy Heuristics

Bartosz Stachowiak 148259<br>
Andrzej Kajdasz 148273

## 1. Problem Statement

There were are columns of integers representing nodes. Each row corresponds to a node and contains its x and y coordinates in a plane, as well as a cost associated with the node. There were 4 such data sets each consisting of 200 rows (each representing a single node).

Problem to solve is to choose precisely 50% of the nodes (rounding up if there is an odd number of nodes) and create a Hamiltonian cycle (a closed path) using this subset of nodes. The goal is to minimize the combined total length of the path and the total cost of the selected nodes.

To calculate the distances between nodes, the Euclidean distance formula was used and then round the results to the nearest integer. As suggested, the distances between the nodes were calculated after loading the data and placed in a matrix, so that during the subsequent evaluation of the problem, it was only necessary to read these values which reduced the cost of the operation of the algorithm.

Three algorithms were used to solve the problem:

- Random Solution
- Nearest Neighbor
- Greedy Cycle

**Note**

As we are optimizing not only for the distance, but also for weight, we have to use a different approach than the one presented in the lecture. We cannot simply choose the nearest neighbor, as it may be too expensive. Instead, we have to choose the cheapest neighbor that is not too far away. We have to use a greedy approach, but we have to be careful not to get stuck in a local minimum.

Similarily in Greedy Cycle we have to consider the weight of the node, not only the distance, when evaluating the increase in cost.

## 2. Pseudocode of all implemented algorithms

### 2.1 Random Solution

```
function random_solution(nodes, seed, target_size):
    set random seed to seed

    indices := [0, 1, ..., len(nodes) - 1]
    random.shuffle(indices)

    solution := indices[:target_size]
    return solution
```
    

### 2.2 Nearest Neighbor

>Note that in our implementation, distance matrix is not passed as an argument, but a class member, calculated before the algorithm is run.\
>For simplification, here we pass it as an argument.

```
function nearest_neighbor_solution(nodes, start_index, target_size, distance_matrix):
    solution := [start_index]
    visited := [false, false, ..., false]
    visited[start_index] := true

    for i in 0..target_size - 1:
        min_cost := infinity
        min_index := -1
        for j in 0..len(nodes) - 1:
            if visited[j]:
                continue
            if distance_matrix[solution[i]][j] < min_cost:
                min_cost := distance_matrix[solution[i]][j] + nodes[j].weight
                min_index := j
        solution.append(min_index)
        visited[min_index] := true

    return solution
```

### 2.3 Greedy Cycle

>Again, distance matrix passed as an argument.

```
function nearest_neighbor_solution(nodes, start_index, target_size, distance_matrix):
    solution := [start_index]
    visited := [false, false, ..., false]
    visited[start_index] := true

    // while solution is not complete, add nodes
    while len(solution) < target_size:

        min_increase := infinity
        min_index := -1
        insert_point := -1

        if len(solution) == 1:
            // find cheapest neighbor

            for i in 0..len(nodes) - 1:
                if visited[i]:
                    continue
                if nodes[i].weight < min_increase:
                    min_increase := nodes[i].cost
                    min_index := i

            solution.append(min_index)
            visited[min_index] := true
            continue

        for i in 0..len(nodes) - 1:
            if visited[i]:
                continue

            // find cheapest place to insert
            for j in 0..len(solution) - 1:
                
                next_idx := (j + 1) % len(solution)
                added_distance := distance_matrix[solution[j]][i] + distance_matrix[i][solution[next_idx]]
                removed_distance := distance_matrix[solution[j]][solution[next_idx]]
                increase := added_distance + nodes[i].weight - removed_distance

                if increase < min_increase:
                    min_increase := increase
                    min_index := i
                    insert_point := j + 1
        
        solution.insert(insert_point, min_index)
        visited[min_index] := true

    return solution
```

## 3. Results of the computational experiments

### 3.1. Code for visualization of the results

In [ ]:
import pathlib
import itertools

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

In [ ]:
DATA_FOLDER = '../data/'
RESULT_FOLDER = f'{DATA_FOLDER}results/'
INSTANCE_FOLDER = f'{DATA_FOLDER}tsp_instances/'

SOLVERS = {
    'r': "Random",
    'n': "Nearest Neighbor",
    'g': "Greedy Cycle",
}
NUM_NODES = 200

instance_files = [path for path in pathlib.Path(INSTANCE_FOLDER).iterdir() if path.is_file()]
instance_names = [path.name.removesuffix('.csv') for path in instance_files]

In [ ]:
def read_instance(path: str) -> np.ndarray[int]:
    with open(path, 'r') as f:
        text = f.read()

    instance = [
        [
            int(cell) for cell in line.split(';') if cell
        ] for line in text.split('\n') if line
    ]
    return np.array(instance)

def read_solution(path: str) -> tuple[np.ndarray[int], int]:
    with open(path, 'r') as f:
        text = f.read()

    solution = [int(num.strip()) for num in text.split('\n') if num]
    return np.array(solution[:-1]), solution[-1]

def plot_solution_for_instance(instance: np.ndarray[int], solution: np.ndarray[int], ax: plt.Axes) -> None:
    solution = np.append(solution, solution[0])
    ax.plot(instance[solution, 0], instance[solution, 1], c='r')
    ax.scatter(instance[:, 0], instance[:, 1], s=instance[:, 2] / 20, c=instance[:, 2])

In [ ]:
instances_data = {
    name: read_instance(f'{INSTANCE_FOLDER}{name}.csv')
    for name in instance_names
}

In [ ]:
instances_solvers_pairs = itertools.product(instances_data.keys(), SOLVERS.keys())

all_results = {}
all_costs = {}
all_stats = {}

for instance, solver in instances_solvers_pairs:
    all_results[instance] = all_results.get(instance, {})
    all_costs[instance] = all_costs.get(instance, {})
    all_stats[instance] = all_stats.get(instance, {})
    costs = []
    paring_results = []
    for idx in range(NUM_NODES):
        solution, cost = read_solution(f'{RESULT_FOLDER}{instance}-{solver}-{idx}.txt')
        paring_results.append(solution)
        costs.append(cost)
    all_results[instance][solver] = np.array(paring_results)
    all_costs[instance][solver] = np.array(costs)
    all_stats[instance][solver] = {
        'mean': np.mean(costs),
        'std': np.std(costs),
        'min': np.min(costs),
        'max': np.max(costs),
    }

In [ ]:
mean_df = pd.DataFrame(all_stats).T
max_df = pd.DataFrame(all_stats).T
min_df = pd.DataFrame(all_stats).T

for column in SOLVERS.keys():
    mean_df[column] = mean_df[column].apply(lambda x: f'{x["mean"]:.0f} ± {x["std"]:.0f}')
    max_df[column] = max_df[column].apply(lambda x: x['max'])
    min_df[column] = min_df[column].apply(lambda x: x['min'])

for df in [mean_df, max_df, min_df]:
    df.rename(columns=SOLVERS, inplace=True)

### 3.2. Visualizations and statistics for all dataset-algorithm pairs

In tabular form we present the Mean, Standard Deviation, Minimum and Maximum of the results of the algorithms for each dataset.

In [ ]:
print("Mean and std of the costs:")
mean_df


In [ ]:
fig, axs = plt.subplots(1, len(instances_data.keys()), figsize=(15, 5), sharey=True)

for idx, instance in enumerate(instances_data.keys()):
    if idx == 0:
        axs[idx].set_ylabel('Cost')
    axs[idx].set_title(instance)
    axs[idx].violinplot(
        [all_costs[instance][solver] for solver in SOLVERS.keys()],
        showmeans=True,
    )
    axs[idx].set_xticks(range(1, len(SOLVERS.keys()) + 1))
    axs[idx].set_xticklabels(SOLVERS.values(), rotation=45, ha='right')

plt.suptitle('Distribution of the costs')
plt.show()

In [ ]:
print("Max of the costs:")
max_df

In [ ]:
print("Min of the costs:")
min_df

In [ ]:
fig, axs = plt.subplots(1, len(instances_data.keys()), figsize=(15, 5), sharey=True)
bar_width = 0.35

for idx, instance in enumerate(instances_data.keys()):
    if idx == 0:
        axs[idx].set_ylabel('Cost')
    axs[idx].set_title(instance)

    axs[idx].bar(
        np.arange(len(SOLVERS.keys())),
        max_df.loc[instance].values,
        width=bar_width,
        label='Maximal cost',
    )
    axs[idx].bar(
        np.arange(len(SOLVERS.keys())) + bar_width,
        min_df.loc[instance].values,
        width=bar_width,
        label='Minimal cost',
    )

    axs[idx].set_xticks(np.arange(len(SOLVERS.keys())) + bar_width / 2)
    axs[idx].set_xticklabels(SOLVERS.values(), rotation=45, ha='right')

plt.suptitle('Maximal cost')
plt.legend()
plt.show()

## 4. Best solutions for all datasets and algorithms

To more easily compare the results, we present the best solutions for each dataset side by side.

The weight of each node is denoted both by its size and color. The bigger and brighter the node, the higher its weight.

In [ ]:
for idx, instance in enumerate(instances_data.keys()):
    fig, axs = plt.subplots(1, 3, figsize=(20, 5))
    for solver_idx, solver in enumerate(SOLVERS.keys()):
        best_instance_idx = np.argmin(all_costs[instance][solver])
        plot_solution_for_instance(instances_data[instance], all_results[instance][solver][best_instance_idx], axs[solver_idx])
        axs[solver_idx].set_title(f'{SOLVERS[solver]}: {all_costs[instance][solver][best_instance_idx]:.0f}')
    fig.suptitle(f'Instance {instance}', fontsize=16, y=1.05)
plt.show()


### Bonus: Visualization of solution search for both heuristics

#### Nearest Neighbor

![nn](../tmp/n-out-prog-a.gif)

#### Greedy Cycle
![gc](../tmp/g-out-prog-a.gif)

## 5. Source Code

[GitHub](https://github.com/Tremirre/ECP)

## 6. Conclusions

Analyzing the results and visualizations, one can come to several conclusions about the algorithms used in the task:

- Greedy methods are much better than the random node selection method. However, among them, there is a noticeable difference between greedy cycle and nearest neighbor.

- The poor results of the random method are very easily explained. With a large number of nodes, the probability of choosing a good solution is very small, hence the poor performance.
  
- Both greedy methods achieved relatively good results, when comparing them to the random method. The Greedy Cycle approach seems to slightly outperform the Nearest Neighbors one. It is mainly due to the fact that at the end of the nearest neighbor algorithm, i.e. when the number of nodes in the path has already reached the required number, the last added node must be merged with the first one. This results in a very large distance being added to the final result (the edge connecting the first and last node added to the solution). The greedy cycle algorithm does not have the above disadvantage because from the very beginning the solution is created as a cycle, taking into account the overall increase in the cost of the solution.

- Relatively low standard deviation of the results obtained in the greedy methods indicates that the algorithms are quite stable, more reliable than the random method.
  
- An important aspect in greedy methods is how the algorithm avoids nodes that have a very high cost. It often chooses vertices that are more distant but have much lower costs - which is clearly visible in the visualizations.
